##  SETUP VIRTUAL MACHINES FOR MULTIPLE USERS
Use Azure Virtual Machine Scale Set (VMSS) and Azure Command Line Interface (Az CLI) to create N virtual machines. After setting up we invoke a setup script on each VM to setup the environment. 



## Azure Resource Setup

In [ ]:
AZURE_SUBSCRIPTION_ID = "your-subscription-id"
LOCATION = "your-resource-location"  # e.g. australiaeast
VM_SIZE = "your-vm-size"  # e.g. Standard_D8_v3
RG_NAME = "your-resource-group-name"
VMSS_NAME = "{}-vmss".format(RG_NAME)
ADMIN_USER_NAME = "vmadmin"
ADMIN_PASSWORD = "nlpbpMlads2019"

# Number of VM instances in a scale set 
NUM_VM = 3

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [ ]:
!az account show

### Create VMSS

In [ ]:
!az group create --name {RG_NAME} --location {LOCATION}

In [ ]:
# Setup public-ip for each vm by using `--public-ip-per-vm` parameter
!az vmss create -g {RG_NAME} -n {VMSS_NAME} --instance-count {NUM_VM} --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --vm-sku {VM_SIZE} --public-ip-per-vm --admin-username {ADMIN_USER_NAME} --admin-password {ADMIN_PASSWORD}

In [ ]:
# Check public ip addresses of the instances
!az vmss list-instance-public-ips --resource-group {RG_NAME} --name {VMSS_NAME} | grep 'ipAddress'

## Create User Accounts and Environment Setup

In [ ]:
SCRIPT = "setup_vm.sh"

In [ ]:
%%writefile setup_vm.sh
cd ~  

# clone repo and install the conda env 
git clone https://github.com/microsoft/nlp-recipes.git

# change permission as we copy this into each user's folder
chmod -R ugo+rwx /root/nlp-recipes

source /data/anaconda/etc/profile.d/conda.sh

cd /root/nlp-recipes
python tools/generate_conda_file.py
conda env create -n nlp_cpu -f nlp_cpu.yaml

conda activate my_env_name
python -m ipykernel install --user --name nlp_cpu --display-name "Python (nlp_cpu)"

# Pip install the utils_nlp package in the conda environment
pip install -e . 


# add users to jupyterhub
echo 'c.Authenticator.whitelist = {"mlads1", "mlads2", "mlads3", "mlads4", "mlads5", "mlads6", "mlads7", "mlads8", "mlads9", "mlads10", "mlads11", "mlads12", "mlads13", "mlads14", "mlads15", "mlads16", "mlads17", "mlads18", "mlads19", "mlads20", "mlads21", "mlads22", "mlads23", "mlads24", "mlads25", "mlads26", "mlads27", "mlads28", "mlads29", "mlads30", "mlads31", "mlads32"}' | sudo tee -a /etc/jupyterhub/jupyterhub_config.py

# create the users on the vm 
for i in {1..32}
do
    USERNAME=mlads$i
    PASSWORD=nlpmlads$i
    sudo adduser --quiet --disabled-password --gecos "" $USERNAME
    echo "$USERNAME:$PASSWORD" | sudo chpasswd
    rm -rf /data/home/$USERNAME/notebooks/*
    
    # copy repo
    cp -ar /root/nlp-recipes /data/home/$USERNAME/notebooks
done

# restart jupyterhub 
sudo systemctl stop jupyterhub 
sudo systemctl start jupyterhub 

exit

In [ ]:
!az vmss list-instances -n {VMSS_NAME} -g {RG_NAME} --query "[].id" --output tsv | az vmss run-command invoke --command-id RunShellScript --scripts @{SCRIPT} --ids @-